<div class="alert alert-block alert-secondary" style="font-size:30px">
EDA Enzyme Stability
</div>

<div class="alert alert-block alert-warning" style="font-size:15px">
Hi dear lector 👀, if this notebook is useful for you, please give us an upvote ❤️🚀
</div>

<div class="alert alert-block alert-info" style="font-size:15px">
Competition Goals
</div>


The goal of this competition is to predict the thermostability of enzyme variants. The experimentally measured thermostability (melting temperature) data includes natural sequences, as well as engineered sequences with single or multiple mutations upon the natural sequences.

Understanding and accurately predict protein stability is a fundamental problem in biotechnology. Its applications include enzyme engineering for addressing the world’s challenges in sustainability, carbon neutrality and more. Improvements to enzyme stability could lower costs and increase the speed scientists can iterate on concepts. ([Novozymes Enzyme Stability Prediction | Kaggle, s. f.](https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction))


Enzymes can be a little complex to understand, so seeing [this video](https://www.youtube.com/watch?v=rlH1ym916Fo) will make you understand better about what are the enzymes in any case you need some visual help.

<div class="alert alert-block alert-info" style="font-size:15px">
What is an enzyme and how it works
</div>


<img src="https://img.freepik.com/vector-premium/diagrama-ilustracion-vectorial-plantilla-ciclo-catalitico-enzimatico_683773-180.jpg?w=740" >

[Image Resource](https://www.freepik.es/vector-premium/diagrama-ilustracion-vectorial-plantilla-ciclo-catalitico-enzimatico_30160592.htm#query=enzyme&position=9&from_view=search&track=sph)


Enzymes are proteins that act as catalysts in the chemical reactions of living organisms, it means that enzymes accelerate reaction speed, modifying substances called substrates, and the substrates which are chosen to bind with the enzymes to be modified, will depend in each enzyme. normally enzymes are proteins but also can be RNA.


<div class="alert alert-block alert-success" style="font-size:13px">
How enzymes speed up chemical reactions?
</div>

Enzymes accomplish this task by reducing the reaction activation energy. The amount of energy required to start a reaction varies depending on the type of enzyme, and this is where we will discuss our objective prediction, "thermostability".

After enzymes binds to the substrate it makes bond breaking and bond forming, binding substrates and creating a new product as result. All this process happens into a special place called "active site", this place has a specific size, behavior and shape.

This shape belongs to the link between the amino acids from the enzyme, the enzymes has these compressed amino acids in one or more polypeptide chains, the size of this chains will define which substrates are "compatible" with the enzyme.

<div class="alert alert-block alert-success" style="font-size:13px">
Enzyme components
</div>

<img src="https://www.learninsta.com/wp-content/uploads/2021/06/Enzymes-img-4.png" >

[Image Resource](https://www.learninsta.com/wp-content/uploads/2021/06/Enzymes-img-4.png)

* Cations:
Positively charged metal irons which act as activators, these cations are added temporarily to the active site to activate the enzyme (named cofactor in the image).
* Coenzymes:
Coenzymes are vitamin products which join to the enzyme-substrate complex temporarily!
* Prosthetic Group:
It is a permanently enzyme bound


All this system is called Holoenzyme and an inactivated enzyme (without the other components), is called inactivated enzyme.

# Defining libraries, paths and constants

In [ ]:
%%capture
!pip install blosum
!pip install biopandas
import sys
# SYS installations


!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
! wget https://ftp.ncbi.nih.gov/blast/matrices/BLOSUM100 -O BLOSUM100.txt
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

!nvidia-smi

# Check CUDA/cuDNN Version
!nvcc -V && which nvcc

In [ ]:
# Base libraries

import numpy as np 
import pandas as pd 
import random
import requests
from biopandas.pdb import PandasPdb
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy.stats import rankdata
from tqdm.auto import tqdm

# Files management
import os

# Ploting
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Libraris for Data process
import blosum as bl
import Levenshtein
from Levenshtein import distance as levenshtein_distance
from Bio.SubsMat import MatrixInfo
#import cudf


* Cudf librarie tutoral: https://www.kaggle.com/code/beniel/rapids-cudf-tutorial
* Levensthein Distance Explanation: https://www.youtube.com/watch?v=MiqoA-yF-0M

In [ ]:
# Paths


## Novozymes Database
testCsvPath = "../input/novozymes-enzyme-stability-prediction/test.csv"
trainCsvPath = "../input/novozymes-enzyme-stability-prediction/train.csv"

sampleSubmissionPath = "../input/novozymes-enzyme-stability-prediction/sample_submission.csv"
trainUpdatesPath = "../input/novozymes-enzyme-stability-prediction/train_updates_20220929.csv"

pdbPath = "/kaggle/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"


## External Databases

trainDFGroupPath = "../input/train-data-contains-mutations-like-test-data/train_with_groups.csv"

# Understanding dataset

<div class="alert alert-block alert-info" style="font-size:15px">
testDF
</div>


In [ ]:
testDF = pd.read_csv(testCsvPath)
testDF.head()

In [ ]:
testDF.info

<div class="alert alert-block alert-info" style="font-size:15px">
trainDF
</div>


In [ ]:
trainDF = pd.read_csv(trainCsvPath)
print(trainDF["data_source"][0])
print(trainDF["data_source"][10])
print(trainDF["data_source"][100])

In [ ]:
trainDF.head()

In [ ]:
len(trainDF["seq_id"])

In [ ]:
trainDF["protein_sequence"][0]

In [ ]:
## As we can see, we have only 325 pages where comes from our data
len(pd.unique(trainDF["data_source"]))

<div class="alert alert-block alert-info" style="font-size:15px">
Sample Submission
</div>


In [ ]:
sampleSubmission = pd.read_csv(sampleSubmissionPath)
sampleSubmission.head()

In [ ]:
sampleSubmission.info

<div class="alert alert-block alert-info" style="font-size:15px">
Train Updates
</div>

In [ ]:
trainUpdates = pd.read_csv(trainUpdatesPath)
trainUpdates

In [ ]:
trainUpdates.info

## Important Technicalities

In the data we can see the next columns: 

* Protein sequence
* pH
* tm
* data_source

Lets explain each column

<div class="alert alert-block alert-info" style="font-size:15px">
pH
</div>

According to Brittanica, pH is a quantitative measure of acidity, basicity of aqueus in liquid solutions([The Editors of Encyclopaedia Britannica, 2022](https://www.britannica.com/science/pH)).

Since pH has an impact on enzyme activity, it is crucial to monitor pH in relation to enzymes. However, there is a certain pH level at which enzyme activity will be at its peak; for example, see the example below.

<img src="https://bam.files.bbci.co.uk/bam/live/content/zskcd2p/large" >

[Image Resource](https://www.bbc.co.uk/bitesize/guides/zcr74qt/revision/5#:~:text=Enzyme%20activity%20is%20at%20its,pH%20the%20enzyme%20activity%20decreases.)

Amino acids present in the active site are acidic or basic and the fluctuation in pH can affect these amino acids, remember that this is so important because amino acids helps to the enzyme to define which substrate are going to be joined, and with pH fluctuations can make harder to the substrates to bind and extreme pH values can denature enzymes.

<div class="alert alert-block alert-info" style="font-size:15px">
tm
</div>

In this case tm is referring to the thermostability measure, it is very important for industrial applications, measuring this have a special method, for better understanding how to measure see [this article](https://iubmb.onlinelibrary.wiley.com/doi/pdf/10.1002/bmb.21127).
Also this is going to be our focus goal prediction, the production as result of enzymes process will depends in a big way of this metric.

<div class="alert alert-block alert-success" style="font-size:13px">
How thermostability affects Enzymes cicle?
</div>

Each enzyme's active site is extremely sensitive, and even a small change in the environment can cause the enzyme to respond differently. Thermoestability is one of the elements that affects the active site and, as a result, how well the enzyme works.

The suitable temperature for enzymes to function properly is 37ºC, increasing or decreasing the temperature above 37ºC can affect chemical bonds in the active site making them less suited to bind substrates and higher temperatures denature enzymes ph. ([Effect of Temperature on Enzymatic Reaction - Creative Enzymes, s. f.](https://www.creative-enzymes.com/resource/effect-of-temperature-on-enzymatic-reaction_50.html#:~:text=The%20optimum%20temperature%20for%20most,enzymes%20adapted%20to%20higher%20temperatures.))



<div class="alert alert-block alert-info" style="font-size:15px">
Protein Sequence
</div>
​
​
Protein sequence is the method used to determinate all the amino acids in a protein (remember enzymes are conformed by proteins or ARN), it is until to identify it in a sample and categorize its post-translational modifications. The process of determining the amino acid sequence is known as protein sequencing. [(Smith, 2019)](https://www.news-medical.net/life-sciences/Amino-Acids-and-Protein-Sequences.aspx)
​
<img src="https://d2jx2rerrg6sh3.cloudfront.net/image-handler/ts/20170523082107/ri/673/picture/2017/5/DNA_translation_to_prtoein_680x_-_udaix.jpg" >
​
[Image Resource](https://d2jx2rerrg6sh3.cloudfront.net/image-handler/ts/20170523082107/ri/673/picture/2017/5/DNA_translation_to_prtoein_680x_-_udaix.jpg)

<div class="alert alert-block alert-info" style="font-size:15px">
Data source
</div>

This column is no complex to explain, each value into each row from this column is going to returned a link which say us where was take the information.

### Amino Acids Metrics


According to Evolutions and genomics, the divergence among sequences can be modeled with a mutation matrix. [(Amino acid substitution models, 2016)](https://evomics.org/resources/substitution-models/amino-acid-substitution-models/#:~:text=Empirical%20substitution%20models&text=In%20the%20Dayhoff%20approach%2C%20replacement,of%20successive%20mutations%20is%20low.).
In that reference we cand find that there are differents ways to see the divergences and mutations in amino acids, between them exists:

* PAM Matrices
* Dayhoff Matrices
* JIT Matrices
* BLOSUM
* Poisson models

Blocks Amino Acid Substitution Matrices is a substition metric used for sequences alignment in proteins, a lecture talking about this deeply is the(Gilbert et al., 1992)

What is the sequence alignment for?

A sequence alignment is a way of arranging the primary sequences of a protein to identify regions of similarity that may be a consequence of functional, structural, or evolutionary relationships between the sequences.([Aligning multiple protein sequences | UniProt, s. f.](https://www.ebi.ac.uk/training/online/courses/uniprot-exploring-protein-sequence-and-functional-info/how-to-use-uniprot-tools-clone/aligning-multiple-protein-sequences/#:~:text=A%20sequence%20alignment%20is%20a,evolutionary%20relationships%20between%20the%20sequences.))

Please see the next video to understand better what are we doing in the next code blocks
https://www.youtube.com/watch?v=MiqoA-yF-0M

<div class="alert alert-block alert-success" style="font-size:13px">
Test Data Mutations
</div>


In [ ]:
# Wild type sequence provided in the "Dataset Description":
base = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

In [ ]:
listOutput = []

In [ ]:
for i in range(len(testDF)):
    rowProteinSequence = testDF.iloc[i].protein_sequence ## Taking a row
    LevenstreinDist = Levenshtein.editops(rowProteinSequence, base) ## Using Levenshtein
    listOutput.append(LevenstreinDist)

In [ ]:
def deal(s):
    if len(s):
        return s[0][2]
    else:
        return -1

In [ ]:
testOutputs = [deal(s) for s in listOutput]
subScores = []

In [ ]:
listOutput[:5]

In [ ]:
testDF['modif'] = testOutputs
testDF.head()

<div class="alert alert-block alert-info" style="font-size:15px">
Protein Data Bank (PDB)
</div>
​
​
Protein Data Bank (PDB) format is a standard for files containing atomic coordinates. Structures deposited in the Protein Data Bank at the Research Collaboratory for Structural Bioinformatics (RCSB) written in this standardized format https://www.wwpdb.org/. While this short description will suffice for many users, for more details, consult the definitive description link: https://www.wwpdb.org/documentation/file-format.

#### Description
Protein Data Bank format consists of lines of information in a text file. Each line of information in the file is called a record. A file generally contains several different types of records, which are arranged in a specific order to describe a structure.



#### Protein Data Bank Record Types
|Record Type||Data Provided by Record|
|-----------||-----------------------|
|ATOM||atomic coordinate record containing the X,Y,Z orthogonal Å coordinates for atoms in standard residues (amino acids and nucleic acids).|
|HETATM||atomic coordinate record containing the X,Y,Z orthogonal Å coordinates for atoms in nonstandard residues. Nonstandard residues include inhibitors, cofactors, ions, and solvent. The only functional difference from ATOM records is that HETATM residues are by default not connected to other residues. Note that water residues should be in HETATM records.|
|TER||indicates the end of a chain of residues. For example, a hemoglobin molecule consists of four subunit chains that are not connected. TER indicates the end of a chain and prevents the display of a connection to the next chain.|
|HELIX||indicates the location and type (right-handed alpha, etc.) of helices. One record per helix.|
|SHEET||indicates the location, sense (anti-parallel, etc.) and registration with respect to the previous strand in the sheet (if any) of each strand in the model. One record per strand.|
|SSBOND||defines disulfide bond linkages between cysteine residues.|


Table above describes the format for selected record types. From the standpoint of most users, the most notable differences between older and newer files occur in the fields following the temperature factor in ATOM and
HETATM records. These fields are not included in the examples in the subsequent sections. Furthermore,
some fields are frequently blank, such as the alternate location indicator when an atom does not have alternate locations.




`Copyright  2002, The Regents of the University of California All Rights Reserved`
​
<img src="https://proteopedia.org/wiki/images/thumb/0/0c/Pdb_file_diagram.png/800px-Pdb_file_diagram.png" >
​
[Image Resource](https://proteopedia.org/wiki/index.php/Image:Pdb_file_diagram.png)

In [ ]:
pdbDF =  PandasPdb().read_pdb(pdbPath)
pdbDF.df.keys()

In [ ]:
atomDF = pdbDF.df['ATOM']
print(f"ATOM dataset is of shape: {atomDF.shape}")

In [ ]:
mapNumberToB = atomDF.groupby('residue_number').b_factor.mean()
testDF['b_factor'] = testDF.modif.map(mapNumberToB).fillna(0)
testDF.head()

In [ ]:
subMat = MatrixInfo.blosum40
list(subMat)[:5]

In [ ]:
for i in range(len(testDF)):
    row = testDF.iloc[i]
    edit = Levenshtein.editops(row.protein_sequence, base)
    if len(edit) == 1:
        if edit[0][0] == 'replace':
            try:
                subScore = subMat[(base[edit[0][2]], row.protein_sequence[edit[0][1]])]
            except KeyError:
                subScore = subMat[(row.protein_sequence[edit[0][1]], base[edit[0][2]])]
        else:
            subScore = -10
    else:
        subScore = -10
    subScores.append(subScore)

B factor is a metric also called as debye waller factor is a metric which take importance for this situacion, see this dicussion to understand better how b factor works:

https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/354476

In [ ]:
testDF['score'] = subScores
testDF['score_adj'] = [ 1 - (1 / (1+np.exp(-x))) for x in subScores]
testDF['bFactorAdj'] = testDF['b_factor'] * testDF['score_adj'] 
testDF.head(10)

In [ ]:
testDF

In [ ]:
def change_list(wild, mutation_list):

    list_output = []

    for mutation in mutation_list:
        edits = Levenshtein.editops(wild, mutation)
        if len(edits):
            list_output.append(edits[0]+tuple(mutation[edits[0][1]])+tuple(base[edits[0][1]]))
        else:
            list_output.append(('replace', 0, 0, 'A', 'A'))

    changes = pd.DataFrame(list_output,columns=['operation','position1','position2','change1','change2'])
    changes.change2 = np.where(changes.operation=='delete','',changes.change2)
    
    return changes

In [ ]:
changes = change_list(base, testDF.protein_sequence.to_list())
changes.operation.value_counts()

In [ ]:
changes

In [ ]:
# Adding changes to the testDF 
test_df = testDF.merge(changes, left_index=True,right_index = True)
test_df

In [ ]:
# Saving test_df as csv
test_df.to_csv('test_data.csv', index=False)

In [ ]:
pd.crosstab(changes.change1,changes.change2).style.background_gradient(axis=None, cmap="YlGnBu")

<div class="alert alert-block alert-info" style="font-size:15px">
Data Mutations
</div>


As is metioned in this [notebook](https://www.kaggle.com/code/cdeotte/train-data-contains-mutations-like-test-data#Train-Data-Contains-Mutations), the data contains mutations, those mutations can be useful to make better predictions but, first we need to add new columns that allow us to identify groups, for that reason we are going to use the database generated by the same notebook.

Some new values saved in the csv are:

* column group which identifies the groups of similar proteins that are mutations



Why we have - 1 groups? Here are some words from the notebook owner:

Before using the new column group with GroupKFold, we must assign every row with group=-1 to a new unique group number (otherwise all the group=-1 will be in the same fold when they are actually different). So for example nrow = train.loc[train.group==-1].shape[0] and then mx = train.group.max() and finally train.loc[train.group==-1,'group'] = np.arange(nrow) + mx + 1.


Also the next code is based in this [notebook](https://www.kaggle.com/code/lucasmorin/nesp-changes-eda-and-baseline) with some modifications.

In [ ]:
trainDFGroup = pd.read_csv(trainDFGroupPath) ## Reading data from train-data-contains-mutations
trainDFGroup

In [ ]:
## Let's see the groups in dataframe
np.unique(trainDFGroup["group"])

In [ ]:
# remove group -1:
trainDF = trainDFGroup[trainDFGroup.group!=-1] ## Avoiding -1 groups
countByGroup = trainDF.groupby('group').seq_id.count() ## Counting proteins by group in dataframe
countByGroup

In [ ]:
def build_change_list(groupDF):
    
    listOutput = []
    groupSize = len(groupDF)
    groupValues = groupDF.values
    
    col = ['pH','data_source','tm','group']

    for i in range(groupSize):
        ## Defining constants
        data1 = groupValues[i]
        seqString1 = data1[1]
        values1  = data1[2:]
        
        
        for j in range(groupSize):
            data2 = groupValues[j]
            seqString2 = data2[1]
            values2  = data2[2:]
            

            ## In this "if", we are taking sure to no make comparations or use editops with the same origin values
            if i != j:
                edits = Levenshtein.editops(seqString1, seqString2)
                levenshteinTuple = tuple([seqString1,seqString2])
                
                ## Checking if edits values does exist
                if len(edits)==1:
                    listOutput.append(levenshteinTuple + edits[0]+ tuple(seqString1[edits[0][1]]) + tuple(seqString2[edits[0][1]]) + tuple(values1) + tuple(values2))
                else:
                    listOutput.append(levenshteinTuple + ('replace', 0, 0, 'A', 'A') + tuple(values1) + tuple(values2))
                    

    changes = pd.DataFrame(listOutput,columns=['protSeq1','protSeq2','operation','position1','position2','change1','change2']+[c+'1' for c in col] + [c+'2' for c in col])
    changes.change2 = np.where(changes.operation=='delete','',changes.change2)
    
    return changes

<div class="alert alert-block alert-success" style="font-size:13px">
Train Data Mutations
</div>


In [ ]:
%%time

top_n = 10 #408
main_groups = countByGroup.sort_values(ascending=False).index[:top_n]

df_list = []

for i in main_groups:
    print(f'group {i}')
    group_df = trainDF[trainDF.group==i]
    df = build_change_list(group_df)
    df_list.append(df)

dfSm = pd.concat(df_list)

In [ ]:
dfSm

In [ ]:
# Converting dfSm to Csv
dfSm.to_csv('train_data.csv', index=False)

In [ ]:
# clean data a bit - same sources, change in protein, same pH and pH not too far of 7

dfClean = dfSm[(dfSm.data_source1 == dfSm.data_source2)&(dfSm.pH1 == dfSm.pH2)]
dfClean = dfClean[(dfClean.pH1>=6) & (dfClean.pH1<=8)]
dfClean = dfClean[dfClean.position1 != 0]

dfClean['target'] = dfClean['tm2'] - dfClean['tm1'] 

print(len(dfClean))
display(pd.crosstab(dfClean.change1,dfClean.change2).style.background_gradient(axis=None, cmap="YlGnBu"))

In [ ]:
dfClean.isnull().sum()

In [ ]:
# Exporting dfClean to csv
dfClean.to_csv('clean_train_data.csv', index=False)

In [ ]:
# avg target by protein changes, independtly of emplacement

avgTarget = dfClean.groupby(['change1','change2']).target.mean()
display(avgTarget.unstack().fillna(0).style.background_gradient(axis=None, cmap="RdYlBu").format('{:.2f}'))

# Extra Techniques

(The next part is a work in progress, in next versions we are adding references)

Based in:
[Notebook](https://www.kaggle.com/code/oxzplvifi/deletion-specific-ensemble)
[Notebook](https://www.kaggle.com/code/awater1223/deletion-specific-ensemble-python)

<div class="alert alert-block alert-info" style="font-size:15px">
Defining Functions and Base
</div>


In [ ]:
# Plot rank distributions
def plot_rank_dist(name, ax, show_del=False):

    sns.kdeplot(
        data=testDf.query('type=="SUB"'),
        x='{}_rank'.format(name),
        bw_adjust=0.3,
        lw=3,
        label='SUB',
        ax=ax,
        color='k'
    )

    ax.vlines(
        testDf.query('type=="DEL"')['{}_rank'.format(name)],
        ax.get_ylim()[0],
        ax.get_ylim()[1],
        lw=5,
        label='DEL',
        color=two_colors[0]
    )

    ax.vlines(
        testDf.query('type=="WT"')['{}_rank'.format(name)],
        ax.get_ylim()[0],
        ax.get_ylim()[1],
        lw=5,
        label='WT',
        color=two_colors[1]
    )

    if show_del:
        sns.kdeplot(
            data=testDf.query('type=="DEL"'),
            x='{}_rank'.format(name),
            bw_adjust=0.3,
            lw=3,
            label='DEL',
            ax=ax,
            color=two_colors[0]
        )

        ax.vlines(
            testDf.query('type=="DEL"')['{}_rank'.format(name)],
            ax.get_ylim()[0],
            ax.get_ylim()[1],
            lw=5,
            label='DEL',
            color=two_colors[0]
        )

    ax.set_xlim(-50,2550)
    ax.set_title('{} rank distribution'.format(name), fontsize=20)
    ax.set_xlabel('{}_rank'.format(name), fontsize=20)
    ax.set_ylabel('Density', fontsize=20)

    ax.tick_params(labelsize=12)
    ax.legend(loc=1)

    return ax

<div class="alert alert-block alert-success" style="font-size:13px">
Mounting Base
</div>


In [ ]:
# Read testing set sequences and pH:
test_df = pd.read_csv('../input/novozymes-enzyme-stability-prediction/test.csv')
two_colors = sns.xkcd_palette(['red', 'bright blue'])


In the next code we are getting data manually as levenstein substitution, What is the difference with prevous method? In this we are saving which sequences does have:

* SUB = Substitution
* DEL = Deletion


We also are adding the next columns:

* WT corresponding to "wildType"
* MUT corresponding to "Mutation"
* resId corresponding to residual Id which indicate us position of mutation (modif contains index)

In [ ]:
# Add mutation information to testing set:
result = []
for _, row in test_df.iterrows():
    ops = Levenshtein.editops(base, row['protein_sequence'])
    assert len(ops) <= 1
    if len(ops) > 0 and ops[0][0] == 'replace':
        idx = ops[0][1]
        result.append(['SUB', idx + 1, base[idx], row['protein_sequence'][idx]])
    elif len(ops) == 0:
        result.append(['WT', 0, '', ''])
    elif ops[0][0] == 'insert':
        assert False, "Ups"
    elif ops[0][0] == 'delete':
        idx = ops[0][1]
        result.append(['DEL', idx + 1, base[idx], '_'])
    else:
        assert False, "Ups"

In [ ]:
testDf = pd.concat([testDF, pd.DataFrame(data=result, columns=['type', 'resid', 'wt', 'mut'])], axis=1)

In [ ]:
testDf.head()

# Blosum

<div class="alert alert-block alert-info" style="font-size:15px">
What is Blosum?
</div>


To understand deeply you can read: https://www.kaggle.com/code/dschettler8845/novo-esp-eli5-performant-approaches-lb-0-451?scriptVersionId=107187300 in any case in future versions we are adding a summarize in this notebook

BLOSUM (Blocks of Amino Acid Substitution Matrix) is a substitution matrix used for sequence alignment of proteins. BLOSUM matrices are used to score alignments between evolutionarily divergent protein sequences. BLOSUM is based on local alignments[(Wiki)](https://wikidoc.org/index.php/BLOSUM#:~:text=BLOSUM%20(BLOcks%20of%20Amino%20Acid,is%20based%20on%20local%20alignments.).

How it is going to be utilize in this case? Simple, BLOSUM (in this case we are using BLOSUM 100) is a table which is going to return us the values depending of the mutation RNA letter

From here we are going to print head dataframe before and after of each technique to understand bether what is goin on.

In [ ]:
testDf.head(1)

In [ ]:
def blosum_apply(row):
    if row['type'] == 'SUB':
        return blosum.loc[row['wt'], row['mut']]
    elif row['type'] == 'DEL':
        return -10
    elif row['type'] == 'WT':
        return 0
    else:
        assert False, "Ups"

In [ ]:
blosum = pd.read_csv('./BLOSUM100.txt', sep='\s+', comment='#')
testDf['blosum'] = testDf.apply(blosum_apply, axis=1)
testDf['blosum_rank'] = rankdata(testDf['blosum'])

In [ ]:
!rm ./BLOSUM100.txt

In [ ]:
testDf.head(1)

In this case we have added blosum and blosum_rank to the dataframe, blosum corresponding to the blosum value per mutation and blosum_rank as a average rank.

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='blosum', ax=ax, show_del=False)
plt.show()

Deletions in this case is giving us no values after using them in BLOSUM, for that reason there can be another methods wich will be more helpful with this situation, and also after seeing it, now we know that we should use more than a method to create predictions.

# pLDDT

<div class="alert alert-block alert-info" style="font-size:15px">
What is pLDDT?
</div>


To understand deeply you can read: https://www.kaggle.com/code/cdeotte/difference-features-lb-0-600 in any case in future versions we are adding a summarize in this notebook

What is a pLDDT score?
The predicted local distance difference test (pLDDT) score (0-100) is a per-residue confidence score, with values greater than 90 indicating high confidence, and values below 50 indicating low confidence.([AlphaFold Error Estimates](https://www.rbvi.ucsf.edu/chimerax/data/pae-apr2022/pae.html))

Video explaining pLDDT visualization: https://www.youtube.com/watch?v=oxblwn0_PMM

In [ ]:
testDf.head(1)

In [ ]:
# Read AlphaFold2 result for wild type sequence:
plddt = (
    pd.read_csv('../input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb', sep='\s+', header=None)[[0,5,10]]
    .rename(columns={0:'atom', 5:'resid', 10:'plddt'})
    .query('atom=="ATOM"')
    .drop_duplicates()
)

# Add B factor to the testing set:
testDf = pd.merge(
    testDf,
    plddt,
    left_on='resid',
    right_on='resid',
    how='left'
)

testDf['plddt_rank'] = rankdata(-1*testDf['plddt'])

In [ ]:
testDf.head(1)

Now we have extra data corresponding to these columns:
* Atom
* plddt
* plddt_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='plddt', ax=ax, show_del=True)
plt.show()

In this case plddt is giving us more information than Blosum100 and now DEl get better values, but we should continue exploring for extra methods.

## Differential pLDDT

<div class="alert alert-block alert-info" style="font-size:15px">
What is pLDDT?
</div>


To understand deeply you can read: https://www.kaggle.com/code/cdeotte/difference-features-lb-0-6000 in any case in future versions we are adding a summarize in this notebook

In [ ]:
testDf.head(1)

In [ ]:
plddtdiff = []

# Wild type result:
wt_plddt = (
    pd.read_csv('../input/nesp-kvigly-test-mutation-pdbs/WT_unrelaxed_rank_1_model_3.pdb', sep='\s+')
    .loc['ATOM'].reset_index()
    .loc[:, ['level_4', 'MODEL']].drop_duplicates()
    .rename(columns={'level_4':'resid', 'MODEL':'plddt'})
    .astype({'resid':int})
    .set_index('resid')
)


In [ ]:
wt  = pd.read_csv('../input/nesp-kvigly-test-mutation-pdbs/A101E_unrelaxed_rank_1_model_3.pdb')

In [ ]:
wt.head()

In [ ]:
wt = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

In [ ]:
# Add difference in pLDDTto the testing set:>
for _,row in tqdm(testDf.iterrows(), total=testDf.shape[0]):
    
    file_path = '../input/nesp-kvigly-test-mutation-pdbs/{}{}{}_unrelaxed_rank_1_model_3.pdb'.format(row['wt'], row['resid'], row['mut'])
    if os.path.exists(file_path):
        tdf = (
            pd.read_csv(file_path, sep='\s+')
            .loc['ATOM'].reset_index()
            .loc[:, ['level_4', 'MODEL']].drop_duplicates()
            .rename(columns={'level_4':'resid', 'MODEL':'plddt'})
            .astype({'resid':int})
            .set_index('resid')
        )
        plddtdiff.append((tdf.loc[row['resid']] - wt_plddt.loc[row['resid']]).values[0])
    else:
        plddtdiff.append(np.nan)

In [ ]:
testDf['plddtdiff'] = plddtdiff
testDf['plddtdiff_rank'] = rankdata(testDf['plddtdiff'])

In [ ]:
testDf.head(1)

Added Columns:
* plddtdiff
* plddtdiff_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='plddtdiff', ax=ax, show_del=True)
plt.show()

# DeepDDG

<div class="alert alert-block alert-info" style="font-size:15px">
What is DeepDDG?
</div>


To understand deeply you can read: https://www.kaggle.com/code/dschettler8845/novo-esp-eli5-performant-approaches-lb-0-451?scriptVersionId=107187300 in any case in future versions we are adding a summarize in this notebook

Accurately predicting changes in protein stability due to mutations is important for protein engineering and for understanding the functional consequences of missense mutations in proteins. We have developed DeepDDG, a neural network-based method, for use in the prediction of changes in the stability of proteins due to point mutations. The neural network was trained on more than 5700 manually curated experimental data points and was able to obtain a Pearson correlation coefficient of 0.48–0.56 for three independent test sets, which outperformed 11 other methods. Detailed analysis of the input features shows that the solvent accessible surface area of the mutated residue is the most important feature, which suggests that the buried hydrophobic area is the major determinant of protein stability. We expect this method to be useful for large-scale design and engineering of protein stability. ([DeepDDG, Huali Chao](https://pubs.acs.org/doi/10.1021/acs.jcim.8b00697))

In [ ]:
testDf.head(1)

In [ ]:
# Run DeepDDG on http://protein.org.cn/ddg.html by uploading the PDB file and clicking "Submit":>
ddg = pd.read_csv('../input/novozymes/ddgout.txt', sep='\s+', usecols=[0,1,2,3,4]).rename(columns={'WT':'wt', 'ResID':'resid', 'Mut':'mut'})

# Add DeepDDG output to the testing set:
testDf = pd.merge(
    testDf.set_index(['wt','resid','mut']),
    ddg.set_index(['wt','resid','mut']),
    left_index=True,
    right_index=True,
    how='left'
).reset_index()

testDf.loc[testDf['type']=='WT','ddG'] = 0
testDf.loc[testDf['type']=='DEL','ddG'] = testDf['ddG'].dropna().median()

testDf['ddG_rank'] = rankdata(testDf['ddG'])

In [ ]:
testDf.head(1)

Added Columns:
* #chain
* ddG
* ddG_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='ddG', ax=ax, show_del=False)
plt.show()

# DeMaSk

<div class="alert alert-block alert-info" style="font-size:15px">
What is DeMaSK?
</div>

To understand deeply you can read: https://www.kaggle.com/code/dschettler8845/novo-esp-eli5-performant-approaches-lb-0-451?scriptVersionId=107187300 in any case in future versions we are adding a summarize in this notebook

DeMaSk uses a simple linear model to predict the functional impact of single amino acid substitutions. It uses an interpretable, directional amino acid substitution matrix computed from deep mutational scanning datasets, as well as per-position evolutionary conservation and variant frequency computed from homologs of the query sequence

In [ ]:
testDf.head(1)

In [ ]:
# Run DeMaSk on https://demask.princeton.edu/query/ by pasting the wild type sequence and clicking "Compute":
demask = pd.read_csv('../input/novozymes/demaskout.txt', sep='\t', usecols=[0,1,2,3], names=['resid','wt','mut','demask'], skiprows=1)

# Add DeMask output to the testing set:
testDf = pd.merge(
    testDf.set_index(['wt','resid','mut']),
    demask.set_index(['wt','resid','mut']),
    left_index=True,
    right_index=True,
    how='left'
).reset_index()

testDf.loc[testDf['type']=='WT','demask'] = 0
testDf.loc[testDf['type']=='DEL','demask'] = testDf['demask'].dropna().min()


testDf['demask_rank'] = rankdata(testDf['demask'])

In [ ]:
testDf.head(1)

Added Columns:
* demask
* demask_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='demask', ax=ax, show_del=False)
plt.show()

#  RMSD

<div class="alert alert-block alert-info" style="font-size:15px">
What is DeMaSK?
</div>

The rmsd value gives the average deviation between the corresponding atoms of two proteins: the smaller the rmsd, the more similar the two structures. Efficient algorithms have been developed to find the best orientation of two structures that gives the minimal possible rmsd [2], [3] [scienceDirec](https://www.sciencedirect.com/science/article/pii/S1359027898000194).

To understand deeply you can read: https://www.kaggle.com/code/oxzplvifi/rmsd-from-molecular-dynamics in any case in future versions we are adding a summarize in this notebook



In [ ]:
testDf.head(1)

In [ ]:
# Read VMD/NAMD output:
namd = pd.read_csv('../input/novozymes-md2/residue_rmsd_sasa_last.dat', sep='\t', header=None, names=['resid','rmsd','sasa0','sasaf'])

# Add VMD/NAMD results to the testing set:
testDf = pd.merge(
    testDf,
    namd[['resid','rmsd']],
    left_on='resid',
    right_on='resid',
    how='left'
)

testDf.loc[testDf['type']=='WT','rmsd'] = testDf['rmsd'].dropna().max()
# test_df.loc[test_df['type']=='WT','sasaf'] = test_df['sasaf'].dropna().max()

testDf['rmsd_rank'] = rankdata(testDf['rmsd'])
# test_df['sasaf_rank'] = rankdata(test_df['sasaf'])

In [ ]:
testDf.head(1)

Added Columns:
* rmsd
* rmsd_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='rmsd', ax=ax, show_del=True)
plt.show()

# SASA

<div class="alert alert-block alert-info" style="font-size:15px">
What is DeMaSK?
</div>

Solvent accessible surface area (SASA) of proteins has always been considered as a decisive factor in protein folding and stability studies. It is defined as the surface characterized around a protein by a hypothetical centre of a solvent sphere with the van der Waals contact surface of the molecule ([S. Ausaf](https://pubmed.ncbi.nlm.nih.gov/24678666/#:~:text=Solvent%20accessible%20surface%20area%20(SASA,contact%20surface%20of%20the%20molecule.)).

To understand deeply you can read: https://www.kaggle.com/code/oxzplvifi/rmsd-from-molecular-dynamics in any case in future versions we are adding a summarize in this notebook

In [ ]:
testDf.head(1)

In [ ]:
namd

In [ ]:
# Read VMD/NAMD output:
namd = pd.read_csv('../input/novozymes-md/residue_rmsd_sasa_last.dat', sep='\t', header=None, names=['resid','rmsd','sasa0','sasaf'])

# Add VMD/NAMD results to the testing set:
testDf = pd.merge(
    testDf,
    namd[['resid','sasaf']],
    left_on='resid',
    right_on='resid',
    how='left'
)

# test_df.loc[test_df['type']=='WT','rmsd'] = test_df['rmsd'].dropna().max()
testDf.loc[testDf['type']=='WT','sasaf'] = testDf['sasaf'].dropna().max()

# test_df['rmsd_rank'] = rankdata(test_df['rmsd'])
testDf['sasaf_rank'] = rankdata(testDf['sasaf'])

In [ ]:
testDf.head(1)

Added Columns:
* sasaf
* sasaf_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='sasaf', ax=ax, show_del=True)
plt.show()

# Rosetta

<div class="alert alert-block alert-info" style="font-size:15px">
What is DeMaSK?
</div>

The Rosetta software suite includes algorithms for computational modeling and analysis of protein structures. It has enabled notable scientific advances in computational biology, including de novo protein design, enzyme design, ligand docking, and structure prediction of biological macromolecules and macromolecular complexes.([ROSETTA](https://www.rosettacommons.org/software))

To understand deeply you can read: https://www.kaggle.com/code/shlomoron/nesp-relaxed-rosetta-scores in any case in future versions we are adding a summarize in this notebook

In [ ]:
testDf.head(1)

In [ ]:
testDf['rosetta_rank'] = pd.read_csv('../input/nesp-relaxed-rosetta-scores/submission_rosetta_scores')['tm']

In [ ]:
testDf.head(1)

Added Columns:

* rosseta_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='rosetta', ax=ax, show_del=False)
plt.show()

# Thermonet

<div class="alert alert-block alert-info" style="font-size:15px">
What is DeMaSK?
</div>

ThermoNet is a computational framework for quantitative prediction of the changes in protein thermodynamic stability () caused by single-point amino acid substitutions. The core algorithm of ThermoNet is an ensemble of deep 3D convolutional neural networks([reference](https://bio.tools/thermonet-protein#:~:text=ThermoNet%20(biotools%3Athermonet%2Dprotein)&text=ThermoNet%20is%20a%20computational%20framework,deep%203D%20convolutional%20neural%20networks.)).

To understand deeply you can read: https://www.kaggle.com/code/vslaykovsky/nesp-thermonet-v2 in any case in future versions we are adding a summarize in this notebook

In [ ]:
testDf.head(1)

In [ ]:
testDf['thermonet'] = pd.read_csv('../input/nesp-thermonet-v2/submission.csv')['tm']
testDf['thermonet_rank'] = rankdata(testDf['thermonet'])

In [ ]:
testDf.head(1)

Added Columns:

* thermonet
* thermonet_rank

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='thermonet', ax=ax, show_del=False)
plt.show()

# 3D Properties

In [ ]:
testDf['p3'] = pd.read_csv('/kaggle/input/nesp-submissions/submission_P03_0482.csv')['tm']
testDf['p3_rank'] = rankdata(testDf['p3'])

fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='p3', ax=ax, show_del=False)
plt.show()

In [ ]:
testDf['p6'] = pd.read_csv('/kaggle/input/nesp-submissions/submission_P06_0460.csv')['tm']
testDf['p6_rank'] = rankdata(testDf['p6'])

fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='p6', ax=ax, show_del=False)
plt.show()

In [ ]:
testDf['p2'] = pd.read_csv('/kaggle/input/nesp-submissions/submission_P02_0448.csv')['tm']
testDf['p2_rank'] = rankdata(testDf['p2'])

fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='p2', ax=ax, show_del=False)
plt.show()

testDf['3D_avg_tn'] = pd.read_csv('/kaggle/input/nesp-submissions/submission_TN_0488.csv')['tm']
testDf['3D_avg_tn_rank'] = rankdata(testDf['3D_avg_tn'])

fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='3D_avg_tn', ax=ax, show_del=False)
plt.show()

> # RASP

In [ ]:
testDf['rasp'] = pd.read_csv('/kaggle/input/nesp-submissions/submission_rasp_0437.csv')['tm']
testDf['rasp_rank'] = rankdata(testDf['rasp'])

fig, ax = plt.subplots(figsize=(25, 5))
plot_rank_dist(name='rasp', ax=ax, show_del=False)
plt.show()

# Ensembling

As we have seen in previous techniques each of them have different accuracys for each mutation case, so for that same reason in the next code is going to show us an algorithm wich mixes all thoses previous values.

In [ ]:
testDf.head(1)

In [ ]:
def rank_nrom(name):
    # Name is a value corresponding to each previous column added so yes, its is beeing saved into "s"
    s = testDf['{}_rank'.format(name)]
    # Here is beeing returned a percentage result of the operation
    return s/s.max()

In [ ]:
# Scale ranks prior to ensembling:
# Global ensemble:
testDf['tm'] = (
    4 * rank_nrom('rosetta') + 2*rank_nrom('rmsd') + 2*rank_nrom('thermonet') + 2*rank_nrom('plddtdiff') +\
    rank_nrom('sasaf') + rank_nrom('plddt') + rank_nrom('demask') + rank_nrom('ddG') + rank_nrom('blosum')
) / 14


In [ ]:
# Scale ranks prior to ensembling:
# Global ensemble:
testDf['tm'] = (
    4 * rank_nrom('rosetta') + 2*rank_nrom('rmsd') + 2*rank_nrom('thermonet') + 2*rank_nrom('plddtdiff') +\
    rank_nrom('sasaf') + rank_nrom('plddt') + rank_nrom('demask') + rank_nrom('ddG') + rank_nrom('blosum') +\
    rank_nrom('rasp') + 0*rank_nrom('3D_avg_p3')
) / 16


In [ ]:
submits = ['rosetta', 'rmsd', 'thermonet', 'plddtdiff', 'sasaf', 'plddt', 'demask', 'ddG', 'blosum', 'rasp', 'p3', 'p2', 'p6']
print('\t', end='')
for s1 in submits:
    print(s1[:5],'\t', end='')
print('')
for s1 in submits:
    print(s1[:5],'\t',end='')
    for s2 in submits:
        tm1 = testDf[f'{s1}_rank']
        tm2 = testDf[f'{s2}_rank']
        print(f"{spearmanr(tm1,tm2).correlation:.4f}\t",end = '')
    print('')
        
        
        
        


In [ ]:
# Scale ranks prior to ensembling:
# Global ensemble:
#testDf['tm'] = (
#    4 * rank_nrom('rosetta') + 2*rank_nrom('rmsd') + 2*rank_nrom('thermonet') + 2*rank_nrom('plddtdiff') +\
#    rank_nrom('sasaf') + rank_nrom('plddt') + rank_nrom('demask') + rank_nrom('ddG') + rank_nrom('blosum') +\        
#    2.5*rank_nrom('3D_avg_p3')
#) / 14


In [ ]:
# Scale ranks prior to ensembling:
# Global ensemble:
#testDf['tm'] = (
#    4 * rank_nrom('rosetta') + 2*rank_nrom('rmsd') + 2*rank_nrom('thermonet') + 2*rank_nrom('plddtdiff') +\
#    rank_nrom('sasaf') + rank_nrom('plddt') + rank_nrom('demask') + rank_nrom('ddG') + rank_nrom('blosum') +\
#    2*rank_nrom('3D_avg_p2')
#) / 14

In [ ]:
testDf.head()

In [ ]:
# Dividing each row values per max in column
testDf['tm'] = testDf['tm']/testDf['tm'].max()

In [ ]:
testDf.head()

In [ ]:
# Deletion type:
idx = testDf[testDf['type']=="DEL"].index
testDf.loc[idx, 'tm'] =  (2*rank_nrom('plddt')[idx] + 2*rank_nrom('plddtdiff')[idx] + rank_nrom('rmsd')[idx] + rank_nrom('sasaf')[idx]) / 6

In [ ]:
# Wild type:
testDf.loc[testDf['type']=="WT",'tm'] = testDf['tm'].max()+1
testDf['tm'] = rankdata(testDf['tm'])
testDf['tm_rank'] = testDf['tm']

In [ ]:
# Submission:
testDf[['seq_id','tm']].to_csv('submission.csv', index=False)

In [ ]:
from scipy.stats import spearmanr

df_0613 = pd.read_csv('/kaggle/input/nesp-submissions/submission_0613.csv')
df_0482 = pd.read_csv('/kaggle/input/nesp-submissions/submission_P03_0482.csv')
df_0448 = pd.read_csv('/kaggle/input/nesp-submissions/submission_P02_0448.csv')

print('sp with 0.613: ', spearmanr(testDf.tm, df_0613.tm))

In [ ]:
diffs = rankdata(df_0448.tm[50:80]) - rankdata(df_0482.tm[50:80])

plt.plot(diffs)

In [ ]:
diffs = rankdata(df_0482.tm[:100]) #- rankdata(df_0482.tm)

plt.plot(diffs)

In [ ]:
testDf.head(1)

How do you select coefficients for each method in the global ensemble? Based on what?

Thank you, good question, my understanding from reading discussions is that the cv and lb correlation is not very stable, so I simply added +1 to the coefficients of each method until the public score stopped improving. Considering that the public score uses 50% of the test data and that 100% of the test data comes from a single protein, I am hoping this will not cause a significant overfitting to the public half of the test data; although it might still be advisable if possible to try optimizing the coefficents on both the cv and the lb separately for each of the two submissions allowed. I have not attempted yet applying all of the methods to the full training set; but this seems like a promising next step([OscarVillaReal](https://www.kaggle.com/oxzplvifi)).

In [ ]:
    
plot_rank_dist(name='tm', ax=ax, show_del=True)
plt.show()

<div class="alert alert-block alert-success" style="font-size:13px">
Ploting
</div>


In [ ]:
fig, axs = plt.subplots(nrows=10, figsize=(20,40), gridspec_kw={'hspace':0.5})

plot_rank_dist(name='rosetta', ax=axs[0], show_del=False)
plot_rank_dist(name='rmsd', ax=axs[1], show_del=True)
plot_rank_dist(name='thermonet', ax=axs[2], show_del=False)
plot_rank_dist(name='plddtdiff', ax=axs[3], show_del=True)
plot_rank_dist(name='sasaf', ax=axs[4], show_del=True)
plot_rank_dist(name='plddt', ax=axs[5], show_del=True)
plot_rank_dist(name='demask', ax=axs[6], show_del=False)
plot_rank_dist(name='ddG', ax=axs[7], show_del=False)
plot_rank_dist(name='blosum', ax=axs[8], show_del=False)

plot_rank_dist(name='tm', ax=axs[9], show_del=True)

plt.show()

In [ ]:
# Training Model:
### In process, we will adding link at the final of the competition

<div class="alert alert-block alert-warning" style="font-size:15px">
Thank you for reading ❤️🚀
</div>

# References



Gilbert, W., Howard Hughes Medical Institute, Basic Sciences Division, Fred Hutchinson Cancer Research Center & Seattle. (1992, noviembre). Amino acid substitution matrices from protein blocks. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC50453/pdf/pnas01096-0363.pdf

FuseSchool - Global Education. (2017, July 19). Enzymes | Cells | Biology | FuseSchool [Video]. YouTube. https://www.youtube.com/watch?v=rlH1ym916Fo

Novozymes Enzyme Stability Prediction | Kaggle. (n.d.). https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction

Back To Back SWE. (2019, January 11). Edit Distance Between 2 Strings - The Levenshtein Distance (“Edit Distance” on LeetCode) [Video]. YouTube. https://www.youtube.com/watch?v=MiqoA-yF-0M


Back To Back SWE. (2019, January 11). Edit Distance Between 2 Strings - The Levenshtein Distance (“Edit Distance” on LeetCode) [Video]. YouTube. https://www.youtube.com/watch?v=MiqoA-yF-0M

RAPIDs cudf tutorial. (2020, June 18). Kaggle. https://www.kaggle.com/code/beniel/rapids-cudf-tutorial

The Editors of Encyclopaedia Britannica. (2022, November 9). PH | Definition, Uses, & Facts. Encyclopedia Britannica. https://www.britannica.com/science/pH

Effect of Temperature on Enzymatic Reaction - Creative Enzymes. (n.d.). https://www.creative-enzymes.com/resource/effect-of-temperature-on-enzymatic-reaction_50.html

Amino acid substitution models. (2016, July 3). Evolution and Genomics. https://evomics.org/resources/substitution-models/amino-acid-substitution-models/

Aligning multiple protein sequences | UniProt. (n.d.). https://www.ebi.ac.uk/training/online/courses/uniprot-exploring-protein-sequence-and-functional-info/how-to-use-uniprot-tools-clone/aligning-multiple-protein-sequences/

wwPDB.org. (n.d.). wwPDB: File Format. https://www.wwpdb.org/documentation/file-format

Novozymes Enzyme Stability Prediction | Kaggle. (n.d.-b). https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/354476

Train Data Contains Mutations Like Test Data! (2022, September 26). Kaggle. https://www.kaggle.com/code/cdeotte/train-data-contains-mutations-like-test-data

NESP: changes EDA and baseline. (2022, October 4). Kaggle. https://www.kaggle.com/code/lucasmorin/nesp-changes-eda-and-baseline